In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

---
## watchlistのcsvをjsonに変換する


In [2]:
def create_wachlist_json():
    df= pd.read_csv('watchlist/watchlist.csv')

    # jsonファイルに変換してファイルに保存
    json_data = df.to_json(orient='records', force_ascii=False)
    with open ("watchlist/watchlist.json", "w", encoding="utf-8") as json_file:
        json_file.write(json_data)

---
## 期待通りスクレイピングできるか試す関数

In [3]:
def test_get_head(url, selector):
    response_test =requests.get(url)
    soup_test = BeautifulSoup(response_test.content, 'html.parser')
    headline_test = soup_test.select(selector)
    print(headline_test)

# test_get_head(
#     "https://www3.nhk.or.jp/news/catnew.html",
#     "#main > article > section > div > ul > li:nth-child(1) > dl > dd > a"
# )

---
## watchlistの一覧を元に見出しを見に行く

In [9]:
# 現状の一覧を開く（リセットする時はwatchlistをひらく）
# with open('./watchlist/watchlist.json', 'r') as json_file:
with open('./get_headers.json', 'r') as json_file:
    websites = json.load(json_file)

# サイト毎に見出しを見に行き、前回との差分を見る
for site in websites:
    response =requests.get(site["url"])
    soup = BeautifulSoup(response.content, 'html.parser')
    headlines = soup.select(site["selector"])
    
    # 前回の見出しを保存しておく
    site["get_title_before"] = site["get_title"]
    # 取得した見出しを保存
    site["get_title"] = headlines[0].text.strip()

    # 結果を表示
    print(
        site["head"] +
        "\nNew title\n" +
        str(site["get_title"]) + 
        "\nBefore title\n" +
        str(site["get_title_before"])
    )

    # 差分のが出たかを見て、値として保存
    if site["get_title_before"] != site["get_title"]:
        print("Update\n")
        site["update"] = True
    else:
        print("No update\n")        
        site["update"] = False

# 処理の結果を保存
with open("./get_headers.json", "w", encoding="utf-8") as json_file:
    json.dump(websites, json_file, ensure_ascii=False, indent=4)

日経電子版 速報
New title
中国不動産の万科、ムーディーズが2段階格下げ
Before title
中国不動産の万科、ムーディーズが2段階格下げ
No update

NHKニュース 速報・新着一覧
New title
J133節 2位マリノス引き分け あすヴィッセル勝てばリーグ優勝
Before title
中国 6か国の短期滞在 ビザ免除措置実施へ 日本は再開求める
Update

Yahoo!ニュース 速報
New title
【フィギュア】宇野昌磨ＳＰ冒頭の「アイ・ラブ・ユーはどんな意味？」　海外メディアが変化球質問
Before title
「4日間休戦」開始、自宅目指すガザ住民
Update



---
## アップデートがあったものだけ抽出して、jsonの配列に追加して保存

In [10]:
from datetime import datetime
import pytz

# スクレイピングの結果から、アップデートがあったものだけ配列に追加
updates = []
for web in websites:
    if web["update"] == True:
        updates.append(web)

# 一つ以上アップデートがある場合、ファイルを上書きして保存
if 0 < len(updates):
    print("get updates")
    japan_tz = pytz.timezone('Asia/Tokyo')
    current_time_japan = datetime.now(japan_tz)
    formatted_time = current_time_japan.strftime('%Y/%m/%d %H:%M')

    new_object = {
        "date": formatted_time,
        "updates": updates
    }

    # 現状を読み込んで変数に格納
    with open ("update.json", "r") as update:
        update_json = json.load(update)

    # 更新分のオブジェクトを配列を追加して、jsonファイルとして保存
    with open ("update.json", "w") as update:
        update_json.append(new_object)
        json.dump(update_json, update, ensure_ascii=False, indent=4)
else:
    print("no update")

update_json[len(update_json)-1]

get updates


{'date': '2023/11/24 22:32',
 'updates': [{'head': 'NHKニュース 速報・新着一覧',
   'media': 'NHK',
   'url': 'https://www3.nhk.or.jp/news/catnew.html',
   'selector': '#main > article > section > div > ul > li:nth-child(1) > dl > dd > a > em',
   'get_title': 'J133節 2位マリノス引き分け あすヴィッセル勝てばリーグ優勝',
   'get_title_before': '中国 6か国の短期滞在 ビザ免除措置実施へ 日本は再開求める',
   'update': True},
  {'head': 'Yahoo!ニュース 速報',
   'media': 'Yahoo!',
   'url': 'https://news.yahoo.co.jp/flash',
   'selector': '#contentsWrap > div:nth-child(3) > div.sc-dMCwnC.lipeMg > div > a > p',
   'get_title': '【フィギュア】宇野昌磨ＳＰ冒頭の「アイ・ラブ・ユーはどんな意味？」\u3000海外メディアが変化球質問',
   'get_title_before': '「4日間休戦」開始、自宅目指すガザ住民',
   'update': True}]}